# Performance comparison
Presented by Chiyoung Ahn (https://github.com/chiyahn)

Consider minimizing following objective function
$$
f(x) = \sum_{i=1}^N \left[ (x_i - m_i)^2 + \log(x_i) \right]
$$
with a fixed $m$ and some fairly large `N` by limited-memory BFGS (LBFGS):

In [8]:
using NLopt, BenchmarkTools, ForwardDiff, NLSolversBase, DiffResults, Flux

In [9]:
N = 10000
m = range(2,5,length = N)
f(x) = sum((x .- m).^2) + sum(log.(x))

f (generic function with 1 method)

## 1. Vanilla `ForwardDiff.jl`

In [10]:
function g!(G::Vector, x::Vector)
    ForwardDiff.gradient!(G, f, x)
end

function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        g!(grad, x)
    end
    f(x)
end
# define the optimization problem
opt = Opt(:LD_LBFGS, N) # 2 indicates the length of `x`
lower_bounds!(opt, fill(1.0, N)) # find `x` above 0
min_objective!(opt, fg!) # specifies that optimization problem is on minimization

# solve the optimization problem
(minf,minx,ret) = @btime optimize(opt, fill(1.0, N))
numevals = opt.numevals # the number of function evaluations
println("got $minf after $numevals iterations (returned $ret)")

  2.719 s (134741 allocations: 14.77 GiB)
got 11933.964915391838 after 9 iterations (returned SUCCESS)


## 2. `NLoptAdapter` with autodiff

In [15]:
# See nlopt-tutorial.ipynb
struct NLoptAdapter{T} <: Function where T <: AbstractObjective
    nlsolver_base::T
end

# implement fg!; note that the order is reversed
(adapter::NLoptAdapter)(x, df) = adapter.nlsolver_base.fdf(df, x)
(adapter::NLoptAdapter)(result, x, jacobian_transpose) = adapter.nlsolver_base.fdf(result, jacobian_transpose', x)

# constructors
NLoptAdapter(f, x, autodiff) = NLoptAdapter(OnceDifferentiable(f, x, autodiff = autodiff))
NLoptAdapter(f!, x::Vector, F::Vector, autodiff) = NLoptAdapter(OnceDifferentiable(f!, x, F, autodiff = autodiff))

NLoptAdapter

In [12]:
f_opt = NLoptAdapter(f, zeros(N), :forward)

# define the optimization problem
opt = Opt(:LD_LBFGS, N) # 2 indicates the length of `x`
lower_bounds!(opt, fill(0.0, N)) # find `x` above -2.0
min_objective!(opt, f_opt) # specifies that optimization problem is on minimization

# solve the optimization problem
(minf,minx,ret) = @btime optimize(opt, fill(1.0, N))
numevals = opt.numevals # the number of function evaluations
println("got $minf after $numevals iterations (returned $ret)")

  2.385 s (134615 allocations: 14.76 GiB)
got 11933.964915391838 after 9 iterations (returned SUCCESS)


## 3. `NLoptAdapter` with numerical gradients

In [13]:
f_opt = NLoptAdapter(f, zeros(N), :finite)

# define the optimization problem
opt = Opt(:LD_LBFGS, N) # 2 indicates the length of `x`
lower_bounds!(opt, fill(0.0, N)) # find `x` above -2.0
min_objective!(opt, f_opt) # specifies that optimization problem is on minimization

# solve the optimization problem
(minf,minx,ret) = @btime optimize(opt, fill(1.0, N))
numevals = opt.numevals # the number of function evaluations
println("got $minf after $numevals iterations (returned $ret)")

  21.224 s (3295091 allocations: 29.88 GiB)
got 11933.964915391842 after 10 iterations (returned FTOL_REACHED)


## 4. Mighty `Flux.jl`

In [14]:
using Flux
using Flux.Tracker: gradient_

function fg!(x::Vector, grad::Vector)
    val = f(x)
    grad[:] = gradient_(f, x)[1]
    return val
end
# define the optimization problem
opt = Opt(:LD_LBFGS, N) # 2 indicates the length of `x`
lower_bounds!(opt, fill(1.0, N)) # find `x` above 0
min_objective!(opt, fg!) # specifies that optimization problem is on minimization

# solve the optimization problem
(minf,minx,ret) = @btime optimize(opt, fill(1.0, N))
numevals = opt.numevals # the number of function evaluations
println("got $minf after $numevals iterations (returned $ret)")

  9.944 ms (3755 allocations: 10.61 MiB)
got 11933.964915391838 after 9 iterations (returned SUCCESS)
